In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-22 21:34:29--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.69MB/s    in 0.2s    

2021-10-22 21:34:30 (5.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [8]:
# dropna
dropna_df = df.dropna()
dropna_df.count()

2643241

In [11]:
# drop_duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.count()

2643241

In [12]:
# Create the vine_table. DataFrame
vine_df = clean_df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100EO2AQP6B02|          3|            2|          4|   N|                Y|
|R100IITU077KKZ|          5|            0|          0|   N|                Y|
|R100WIAVSCW4D1|          5|            3|          5|   N|                N|
|R1023NXBAJ0BYT|          5|            0|          0|   N|                Y|
|R102CEVVIBXNFB|          1|            0|          2|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [35]:
#filtered 20 or more total votes and helpful_votes is equal to or greater than 50%
df = vine_df.filter((vine_df['total_votes'] >= 20) & (vine_df['helpful_votes'] >= 50))
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20A1J42RKSO1I|          1|           97|        105|   N|                N|
|R2B203I8HHS8MW|          5|           59|         59|   N|                Y|
|R1T12VUVNC8QWM|          1|           99|        103|   N|                Y|
|R1MPFLWI15CRJV|          5|          238|        242|   N|                N|
|R1UPFZJZXFN15U|          5|          246|        250|   N|                Y|
|R395831JJCIG4I|          5|           61|         67|   N|                Y|
|R34R94YQM9UM1Q|          5|           56|         61|   N|                Y|
|R3P0B3XK4OSR2V|          1|           62|         70|   N|                Y|
|R3GDLRGL26NMQG|          5|           55|         60|   N|                Y|
|R1SOPCNW0RV7RB|          5|           57|         60|   N|     

In [37]:
vine_with_yes = df.filter(df.vine == 'Y')
vine_with_yes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1EVWPBBNRGKB2|          5|           65|         67|   Y|                N|
|R1241UCQGRHCNH|          5|           72|         76|   Y|                N|
|R10T07ZEB56X21|          5|          121|        132|   Y|                N|
|R1NRX1AS4TSV2N|          4|           84|         90|   Y|                N|
|R20Z8GS5IVRDVT|          5|          129|        134|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [48]:
vine_with_yes_total = vine_with_yes.count()
vine_with_yes_star5 = vine_with_yes.filter(vine_with_yes.star_rating == 5).count()
vine_with_yes_percentage = vine_with_yes_star5/vine_with_yes_total

print(vine_with_yes_total)
print(vine_with_yes_star5)
print(vine_with_yes_percentage)

55
20
0.36363636363636365


In [38]:
vine_with_no = df.filter(df.vine == 'N')
vine_with_no.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20A1J42RKSO1I|          1|           97|        105|   N|                N|
|R2B203I8HHS8MW|          5|           59|         59|   N|                Y|
|R1T12VUVNC8QWM|          1|           99|        103|   N|                Y|
|R1MPFLWI15CRJV|          5|          238|        242|   N|                N|
|R1UPFZJZXFN15U|          5|          246|        250|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [49]:
vine_with_no_total = vine_with_no.count()
vine_with_no_star5 = vine_with_no.filter(vine_with_no.star_rating == 5).count()
vine_with_no_percentage = vine_with_no_star5/vine_with_no_total

print(vine_with_no_total)
print(vine_with_no_star5)
print(vine_with_no_percentage)

10510
6054
0.576022835394862
